In [48]:
import openai
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from collections import defaultdict

openai.api_type = "azure"
openai.api_base = "https://yash-ams-bot.openai.azure.com/"
openai.api_key = "dd4c31d04b2c480685d34365c0d167c6"
openai.api_version = "2023-07-01-preview"  # Use the latest supported version
OPENAI_API_VERSION = "2024-05-01-preview"
# GPT_DEPLOYMENT_NAME = "ams_gpt-35-turbo"
AZURE_DEPLOYMENT_NAME = "gpt-4o"
EMBEDDING_MODEL_DEPLOYEMENT_NAME = "text-embedding-ada-002"


# Read input data while keeping Number (ID) and combining multiple text columns if needed
def read_excel(file_path, text_columns, id_column="Number"):
    df = pd.read_excel(file_path)
    df = df[[id_column] + text_columns].dropna()  # Keep only relevant columns

    # If multiple text columns, combine them into one
    df["Combined_Text"] = df[text_columns].astype(str).agg(" ".join, axis=1)

    return df[[id_column, "Combined_Text"]]

# Get embeddings from Azure GPT
def get_embeddings(texts):
    response = openai.Embedding.create(
        input=texts,
        engine=EMBEDDING_MODEL_DEPLOYEMENT_NAME
    )
    return [data['embedding'] for data in response['data']]

# Generate cluster names using LLM
def generate_cluster_name(texts):
    prompt = f"These are some text samples from a cluster: {texts[:10]}. What would be a meaningful short name for this category? and only provide short names nothing else"
    
    response = openai.ChatCompletion.create(
        engine=AZURE_DEPLOYMENT_NAME,
        messages=[{"role": "system", "content": "You are an expert in categorizing text data."},
                  {"role": "user", "content": prompt}],
        max_tokens=20
    )
    
    return response['choices'][0]['message']['content'].strip()

# Automatically determine optimal number of clusters
def find_optimal_clusters(embeddings, max_clusters=10):
    scores = []
    cluster_range = range(2, min(max_clusters, len(embeddings)))  # Avoid too many clusters for small data

    for k in cluster_range:
        kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
        labels = kmeans.fit_predict(embeddings)
        score = silhouette_score(embeddings, labels)
        scores.append((k, score))

    # Choose the number of clusters with the highest silhouette score
    best_k = max(scores, key=lambda x: x[1])[0]
    return best_k

# Cluster text data while keeping IDs
def cluster_texts(df, text_column="Combined_Text", id_column="Number", num_clusters=None, batch_size=50):
    all_texts = []
    all_ids = []
    all_embeddings = []

    # Process data in batches
    for i in range(0, len(df), batch_size):
        batch = df.iloc[i:i+batch_size]
        batch_texts = batch[text_column].tolist()
        batch_ids = batch[id_column].tolist()

        embeddings = get_embeddings(batch_texts)  # Ensure this function is defined

        all_texts.extend(batch_texts)
        all_ids.extend(batch_ids)
        all_embeddings.extend(embeddings)

    # Convert embeddings to numpy array
    all_embeddings = np.array(all_embeddings)

    # Determine number of clusters automatically if not provided
    if num_clusters is None:
        computed_clusters = find_optimal_clusters(all_embeddings)  # Ensure this function is defined
        num_clusters = max(5, computed_clusters)  # Ensure at least 2 clusters

    print(f"✅ Using {num_clusters} clusters")

    # Apply K-Means clustering
    kmeans = KMeans(n_clusters=num_clusters, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(all_embeddings)

    # Organize text into clusters while preserving IDs
    clusters = defaultdict(list)
    id_map = defaultdict(list)

    for idx, label in enumerate(cluster_labels):
        clusters[label].append(all_texts[idx])
        id_map[label].append(all_ids[idx])

    # Generate meaningful cluster names
    cluster_names = {cluster_id: generate_cluster_name(texts) for cluster_id, texts in clusters.items()}

    return clusters, cluster_names, id_map

# Save clusters to Excel with IDs
def save_clusters_to_excel(clusters, cluster_names, id_map, output_file):
    data = []
    for cluster_id, texts in clusters.items():
        cluster_name = cluster_names.get(cluster_id, f"Cluster {cluster_id}")
        for i, text in enumerate(texts):
            data.append([id_map[cluster_id][i], cluster_name, text])

    df = pd.DataFrame(data, columns=["Number", "Cluster Name", "Text"])
    df_main = pd.read_excel(input_file)
    df = df_main.merge(df, on="Number")
    df = df.drop(['Text'], axis = 1)
    df.to_excel(output_file, index=False)


# Run the pipeline
input_file = "incident_15Oct24.xlsx"
output_file = "clustered_output.xlsx"
id_column = "Number"

# User can provide one or multiple text columns
text_columns = ["Short description"]  # Example: Change to ["Title", "Description"] to combine multiple columns
num_clusters = 15  # Set to an integer (e.g., 5) to specify clusters, or leave None for auto-detection

df = read_excel(input_file, text_columns, id_column)
clusters, cluster_names, id_map = cluster_texts(df, "Combined_Text", id_column, num_clusters)
save_clusters_to_excel(clusters, cluster_names, id_map, output_file)

print(f"✅ Clustering complete! Output saved to {output_file}")


✅ Using 15 clusters
✅ Clustering complete! Output saved to clustered_output.xlsx


In [51]:
!pip show openai

Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: C:\Users\Nilay\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: aiohttp, requests, tqdm
Required-by: crewai, embedchain, guardrails-ai, instructor, langchain-openai, litellm, llama-index-agent-openai, llama-index-core, llama-index-legacy, mem0ai, pyautogen


In [ ]:
Name: openai
Version: 0.28.0
Summary: Python client library for the OpenAI API
Home-page: https://github.com/openai/openai-python
Author: OpenAI
Author-email: support@openai.com
License: 
Location: C:\Users\Nilay\AppData\Local\Programs\Python\Python311\Lib\site-packages
Requires: aiohttp, requests, tqdm
Required-by: crewai, embedchain, guardrails-ai, instructor, langchain-openai, litellm, llama-index-agent-openai, llama-index-core, llama-index-legacy, mem0ai, pyautogen
WARNING: Ignoring invalid distribution ~otebook (C:\Users\Nilay\AppData\Local\Programs\Python\Python311\Lib\site-packages)
